I am running pixy (for fst, pi, dxy) in a couple of different ways to get an idea of whether nucleotide diversity / divergence and fixation are different at TRD loci than elsewhere.

In [ ]:
stop("Filename shouzld be after 02_VCF_SV as that needs to be done first")

I should be able to get all sites vcf with <https://pixy.readthedocs.io/en/latest/generating_invar/generating_invar.html#generating-allsites-vcfs-using-gatk> and `~/data/trd/GVCF_2489Strains/`

In [15]:
source("../../BrusselSprouts/scripts/functions.R")
scripts_dir="/home/jnrunge/data/trd/mapped_reads/scripts/"
initial_timedate=Sys.time()
library(tidytable)

In [16]:
# make sample name map  samplename\tgvcf

samples=list.files("/home/jnrunge/data/trd/GVCF_2489Strains", "g.vcf.gz$", full.names = TRUE)
samples=data.frame(id=unlist(lapply(basename(samples), getFirst_v2, split=".")),file=samples)
fwrite(samples, "~/data/trd/mapped_reads/truly_all_samples.tsv", col.names = FALSE, sep="\t")

In [17]:
if(!file.exists("~/data/trd/mapped_reads/ALL.DP5-95.chromosome1.DP10.GQ20RGQ20.SNPsRef.vcf.gz")){ # just an example, so that keeping the filtered files on the server is enough
    # unfiltered files are on viseg
    file.create(running_file<-paste0("~/data/TRD/runningGATKFullmerge"))
    cmd="cd ~/data/trd/mapped_reads"# && rm -rf ALL_DB"
    cmd=paste0(cmd, " && gatk GenomicsDBImport --batch-size 200 --genomicsdb-workspace-path ALL_DB --sample-name-map ~/data/trd/mapped_reads/truly_all_samples.tsv -L ~/data/TRD/R64_nucl.fasta.fai.bed")
    cmd=paste0(cmd, " && gatk GenotypeGVCFs --java-options '-Xmx180G' -R ../../TRD/R64_nucl.fasta -all-sites -V gendb://ALL_DB -O ALL.vcf.gz")
    cmd=paste0(cmd, " && bcftools query -l ALL.vcf.gz > ALL.vcf.gz.samples && bcftools view -Ob -o ALL.bcf ALL.vcf.gz && rm -f ~/data/TRD/runningGATKFullmerge")
    execute_cmd_sbatch(cmd, mem="200G", cpu="1", time="long", env="bwaetc", jobname="GATK_merge")

    while(file.exists(running_file)){
        Sys.sleep(60)
    }
}

Now I need to filter and do this carefully. We want to remove indels and low GQ calls. invariant sites have "RGQ" instead of GQ, which is the chance of a wrong call, so here we also want a high number (unlike PL). 

So we want GQ >= 20 or RGQ >= 20 I would think. 

First, we get an overview of the average DP in total, and which positions should be removed because they are outside the 5%-95% range

````bash
bcftools query -f "%CHROM\t%POS\t%INFO/DP\n" ALL.vcf.gz | gzip > ALL.vcf.gz.DP.gz
````


````r
DP=fread("~/data/trd/mapped_reads/ALL.vcf.gz.DP.gz")

print(quantile(as.numeric(DP$V3), c(0.05,0.95), na.rm = TRUE))
````

````bash

# sites filtering depth
bcftools view -i "INFO/DP >= 149240 & INFO/DP <= 408636" -Ob -o ALL.DP5-95.bcf ALL.vcf.gz

bcftools index ALL.DP5-95.bcf

bcftools view ALL.DP5-95.bcf | grep -v ^# | cut -f 1 | uniq > chrs.txt


````

In [18]:
chrs=readLines("~/data/trd/mapped_reads/chrs.txt") # its just a list of chromosomes (see MD block above)
for(c in chrs){
    if(file.exists(paste0("~/data/trd/mapped_reads/ALL.DP5-95.",c,".DP10.GQ20RGQ20.SNPsRef.vcf.gz"))){
        next
    }
    # filtering VCF and splitting into chromosomes
    cmd=paste0('sh -xe ~/TRD/03_GenomicSignals/01_pixy_filter-vcf.sh ',c)
    execute_cmd_sbatch(cmd, mem="8G", cpu="1", time="long", env="bwaetc", jobname="bcftools_filter")
    Sys.sleep(1)
}

In [19]:
samples=readLines("~/data/trd/mapped_reads/ALL.vcf.gz.samples")

In [20]:
# what groups should be run?
pop_files=list()


# each pop
pops=fread("../Shiny/data/Victor/operationalTable_Full2543Sace_Clades.csv")
summary(samples%in%pops$StandardizedName)
popList=left_join(data.table(Strain=samples), select(pops, StandardizedName, Clade), by=c("Strain"="StandardizedName"))
popList=filter(popList, !is.na(Clade))
popList=mutate(popList, Clade=gsub("[ .]","_",Clade))
head(popList)
fwrite(popList, pop_files[["Clades"]]<-"~/data/trd/mapped_reads/ALL.vcf.gz-Clades.popList", sep="\t", col.names = FALSE)

   Mode    TRUE 
logical    2489 

Strain,Clade
<chr>,<chr>
AAA,1__Wine
AAB,8__Belgium_Beer
AAC,10__UK_Beer
AAD,18__Asian_Fermentation
AAE,1__Wine
AAG,16__USA_Clinical_1


In [21]:
selectSimilarity=0.7
df_Strains=fread("../Shiny/data/Victor/operationalTable_Full2543Sace_Clades.csv")

crosses=readLines("~/data/trd/mapped_reads/TRD.vcf.gz.samples")
crosses=crosses[startsWith(crosses, "YJNRC") | startsWith(crosses, "Chris")]
crosses

[1] "ChrisC1" "ChrisC2" "ChrisC3" "ChrisC4" "ChrisC5" "ChrisC6" "ChrisC7"
 [8] "ChrisC8" "YJNRC1"  "YJNRC11" "YJNRC12" "YJNRC14" "YJNRC15" "YJNRC16"
[15] "YJNRC17" "YJNRC18" "YJNRC19" "YJNRC2"  "YJNRC20" "YJNRC21" "YJNRC22"
[22] "YJNRC23" "YJNRC24" "YJNRC25" "YJNRC26" "YJNRC27" "YJNRC29" "YJNRC3" 
[29] "YJNRC30" "YJNRC31" "YJNRC32" "YJNRC34" "YJNRC4"  "YJNRC5"  "YJNRC6" 
[36] "YJNRC7"  "YJNRC8"  "YJNRC9"

In [22]:

# add TRD-similar strains vs rest


for(c in crosses){
    if(!file.exists(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz.allelesharing.csv.gz"))){
        print("skip1")
        next
    }
    if(file.mtime(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz.allelesharing.csv.gz")) < file.mtime(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz"))){
        print("skip2")
        next
    }
    if(file.mtime(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz"))<file.mtime(paste0("/home/jnrunge/data/trd/mapped_reads/TRD.vcf.gz"))){
        print("skip3")
        next
    }
    AS=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz.allelesharing.csv.gz"))

    TRD=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz"))
    
    if(!file.exists(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-TRD_regions.csv.gz"))){
        print("skip4")
        next
    }

    TRD_loci=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-TRD_regions.csv.gz"))
    
    for(i in 1:nrow(TRD_loci)){
        if(TRD_loci$chr_start[i]!=TRD_loci$chr_end[i]){
        stop("chr overlapping TRD")
    }
        TRD_subset=filter(TRD, chr== TRD_loci$chr_start[i] & global_pos >= TRD_loci$global_start[i] & global_pos <= TRD_loci$global_end[i])
        df_AS_filtered=filter(AS, `#CHROM` == TRD_loci$chr_start[i], POS %in% TRD_subset$pos)
        melted=reshape2::melt(df_AS_filtered, id.vars = c("#CHROM","POS"))
        melted=filter(melted, variable != "chrpos")
        tmp=summarise(group_by(melted, variable), nAll=n())
        vcf_translated_summary=left_join(tmp,summarise(group_by(melted, variable, value), n=n()), by=c("variable"))%>%mutate(p=n/nAll)%>%select(variable,value,p)%>%rename(Strain=variable, Type=value)
        A1s=vcf_translated_summary$Strain[vcf_translated_summary$Type=="A1_hom" & vcf_translated_summary$p>=selectSimilarity]
        A2s=vcf_translated_summary$Strain[vcf_translated_summary$Type=="A2_hom" & vcf_translated_summary$p>=selectSimilarity]
        
        
        strain_summary=bind_rows(summarise(group_by(filter(df_Strains, StandardizedName %in% A1s),
                       StandardizedName), n=n()) %>% arrange(-n)%>%mutate(Type="A1_hom"),
              summarise(group_by(filter(df_Strains, StandardizedName %in% A2s),
                                 StandardizedName), n=n()) %>% arrange(-n)%>%mutate(Type="A2_hom"))%>% arrange(-n)
        
        if(mean(TRD_subset$AD_A1/TRD_subset$sumCount)<0.5){
        distorter="A2"
        nondistorter="A1"
    }else{
        distorter="A1"
        nondistorter="A2"}
        
        pop_list_trd=data.frame(sample=samples,pop="other",stringsAsFactors = FALSE)
        pop_list_trd$pop[pop_list_trd$sample%in%strain_summary$StandardizedName[strain_summary$Type==paste0(distorter,"_hom")]]="distorter-like"
        
        
        fwrite(pop_list_trd, pop_files[[paste0("TRD_",c,"_",i,"_",selectSimilarity)]]<-paste0("~/data/trd/mapped_reads/ALL.vcf.gz-",paste0("TRD_",c,"_",i,"_",selectSimilarity),".popList"), sep="\t", col.names = FALSE)
    }
}






[1] "skip4"
[1] "skip4"
[1] "skip4"
[1] "skip4"
[1] "skip4"
[1] "skip4"


In [23]:
names(pop_files)

[1] "Clades"            "TRD_ChrisC1_1_0.7" "TRD_ChrisC1_2_0.7"
  [4] "TRD_ChrisC1_3_0.7" "TRD_ChrisC1_4_0.7" "TRD_ChrisC3_1_0.7"
  [7] "TRD_ChrisC3_2_0.7" "TRD_ChrisC4_1_0.7" "TRD_ChrisC5_1_0.7"
 [10] "TRD_ChrisC5_2_0.7" "TRD_ChrisC7_1_0.7" "TRD_ChrisC7_2_0.7"
 [13] "TRD_ChrisC7_3_0.7" "TRD_ChrisC7_4_0.7" "TRD_ChrisC7_5_0.7"
 [16] "TRD_ChrisC8_1_0.7" "TRD_ChrisC8_2_0.7" "TRD_ChrisC8_3_0.7"
 [19] "TRD_ChrisC8_4_0.7" "TRD_ChrisC8_5_0.7" "TRD_ChrisC8_6_0.7"
 [22] "TRD_YJNRC12_1_0.7" "TRD_YJNRC12_2_0.7" "TRD_YJNRC12_3_0.7"
 [25] "TRD_YJNRC14_1_0.7" "TRD_YJNRC15_1_0.7" "TRD_YJNRC17_1_0.7"
 [28] "TRD_YJNRC17_2_0.7" "TRD_YJNRC17_3_0.7" "TRD_YJNRC18_1_0.7"
 [31] "TRD_YJNRC18_2_0.7" "TRD_YJNRC18_3_0.7" "TRD_YJNRC19_1_0.7"
 [34] "TRD_YJNRC19_2_0.7" "TRD_YJNRC19_3_0.7" "TRD_YJNRC2_1_0.7" 
 [37] "TRD_YJNRC20_1_0.7" "TRD_YJNRC20_2_0.7" "TRD_YJNRC21_1_0.7"
 [40] "TRD_YJNRC21_2_0.7" "TRD_YJNRC22_1_0.7" "TRD_YJNRC22_2_0.7"
 [43] "TRD_YJNRC22_3_0.7" "TRD_YJNRC22_4_0.7" "TRD_YJNRC22_5_0.7"
 [46] "TRD_YJNRC22_6_0.7" "TRD_YJNRC23_1_0.7" "TRD_YJNRC23_2_0.7"
 [49] "TRD_YJNRC24_1_0.7" "TRD_YJNRC24_2_0.7" "TRD_YJNRC24_3_0.7"
 [52] "TRD_YJNRC24_4_0.7" "TRD_YJNRC25_1_0.7" "TRD_YJNRC26_1_0.7"
 [55] "TRD_YJNRC27_1_0.7" "TRD_YJNRC27_2_0.7" "TRD_YJNRC29_1_0.7"
 [58] "TRD_YJNRC3_1_0.7"  "TRD_YJNRC3_2_0.7"  "TRD_YJNRC3_3_0.7" 
 [61] "TRD_YJNRC3_4_0.7"  "TRD_YJNRC30_1_0.7" "TRD_YJNRC30_2_0.7"
 [64] "TRD_YJNRC31_1_0.7" "TRD_YJNRC31_2_0.7" "TRD_YJNRC31_3_0.7"
 [67] "TRD_YJNRC31_4_0.7" "TRD_YJNRC31_5_0.7" "TRD_YJNRC31_6_0.7"
 [70] "TRD_YJNRC32_1_0.7" "TRD_YJNRC4_1_0.7"  "TRD_YJNRC4_2_0.7" 
 [73] "TRD_YJNRC4_3_0.7"  "TRD_YJNRC4_4_0.7"  "TRD_YJNRC4_5_0.7" 
 [76] "TRD_YJNRC4_6_0.7"  "TRD_YJNRC4_7_0.7"  "TRD_YJNRC4_8_0.7" 
 [79] "TRD_YJNRC4_9_0.7"  "TRD_YJNRC4_10_0.7" "TRD_YJNRC4_11_0.7"
 [82] "TRD_YJNRC4_12_0.7" "TRD_YJNRC4_13_0.7" "TRD_YJNRC4_14_0.7"
 [85] "TRD_YJNRC4_15_0.7" "TRD_YJNRC4_16_0.7" "TRD_YJNRC4_17_0.7"
 [88] "TRD_YJNRC4_18_0.7" "TRD_YJNRC4_19_0.7" "TRD_YJNRC4_20_0.7"
 [91] "TRD_YJNRC4_21_0.7" "TRD_YJNRC4_22_0.7" "TRD_YJNRC4_23_0.7"
 [94] "TRD_YJNRC4_24_0.7" "TRD_YJNRC4_25_0.7" "TRD_YJNRC4_26_0.7"
 [97] "TRD_YJNRC4_27_0.7" "TRD_YJNRC4_28_0.7" "TRD_YJNRC4_29_0.7"
[100] "TRD_YJNRC4_30_0.7" "TRD_YJNRC4_31_0.7" "TRD_YJNRC4_32_0.7"
[103] "TRD_YJNRC4_33_0.7" "TRD_YJNRC5_1_0.7"  "TRD_YJNRC5_2_0.7" 
[106] "TRD_YJNRC6_1_0.7"  "TRD_YJNRC6_2_0.7"  "TRD_YJNRC6_3_0.7" 
[109] "TRD_YJNRC6_4_0.7"  "TRD_YJNRC7_1_0.7"  "TRD_YJNRC7_2_0.7" 
[112] "TRD_YJNRC7_3_0.7"  "TRD_YJNRC8_1_0.7"  "TRD_YJNRC8_2_0.7" 
[115] "TRD_YJNRC8_3_0.7"  "TRD_YJNRC8_4_0.7"  "TRD_YJNRC8_5_0.7" 
[118] "TRD_YJNRC9_1_0.7"  "TRD_YJNRC9_2_0.7"

In [24]:
pop_files

$Clades
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-Clades.popList"

$TRD_ChrisC1_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC1_1_0.7.popList"

$TRD_ChrisC1_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC1_2_0.7.popList"

$TRD_ChrisC1_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC1_3_0.7.popList"

$TRD_ChrisC1_4_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC1_4_0.7.popList"

$TRD_ChrisC3_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC3_1_0.7.popList"

$TRD_ChrisC3_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC3_2_0.7.popList"

$TRD_ChrisC4_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC4_1_0.7.popList"

$TRD_ChrisC5_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC5_1_0.7.popList"

$TRD_ChrisC5_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC5_2_0.7.popList"

$TRD_ChrisC7_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC7_1_0.7.popList"

$TRD_ChrisC7_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC7_2_0.7.popList"

$TRD_ChrisC7_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC7_3_0.7.popList"

$TRD_ChrisC7_4_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC7_4_0.7.popList"

$TRD_ChrisC7_5_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC7_5_0.7.popList"

$TRD_ChrisC8_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC8_1_0.7.popList"

$TRD_ChrisC8_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC8_2_0.7.popList"

$TRD_ChrisC8_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC8_3_0.7.popList"

$TRD_ChrisC8_4_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC8_4_0.7.popList"

$TRD_ChrisC8_5_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC8_5_0.7.popList"

$TRD_ChrisC8_6_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_ChrisC8_6_0.7.popList"

$TRD_YJNRC12_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC12_1_0.7.popList"

$TRD_YJNRC12_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC12_2_0.7.popList"

$TRD_YJNRC12_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC12_3_0.7.popList"

$TRD_YJNRC14_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC14_1_0.7.popList"

$TRD_YJNRC15_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC15_1_0.7.popList"

$TRD_YJNRC17_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC17_1_0.7.popList"

$TRD_YJNRC17_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC17_2_0.7.popList"

$TRD_YJNRC17_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC17_3_0.7.popList"

$TRD_YJNRC18_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC18_1_0.7.popList"

$TRD_YJNRC18_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC18_2_0.7.popList"

$TRD_YJNRC18_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC18_3_0.7.popList"

$TRD_YJNRC19_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC19_1_0.7.popList"

$TRD_YJNRC19_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC19_2_0.7.popList"

$TRD_YJNRC19_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC19_3_0.7.popList"

$TRD_YJNRC2_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC2_1_0.7.popList"

$TRD_YJNRC20_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC20_1_0.7.popList"

$TRD_YJNRC20_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC20_2_0.7.popList"

$TRD_YJNRC21_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC21_1_0.7.popList"

$TRD_YJNRC21_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC21_2_0.7.popList"

$TRD_YJNRC22_1_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC22_1_0.7.popList"

$TRD_YJNRC22_2_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC22_2_0.7.popList"

$TRD_YJNRC22_3_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC22_3_0.7.popList"

$TRD_YJNRC22_4_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC22_4_0.7.popList"

$TRD_YJNRC22_5_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC22_5_0.7.popList"

$TRD_YJNRC22_6_0.7
[1] "~/data/trd/mapped_reads/ALL.vcf.gz-TRD_YJNRC22_6_0.7.popList"

$TRD_YJNRC23_1_0.7
[1]

In [25]:
# c is being reused below so should be run after all is prepared above

In [26]:
library(stringr)

In [27]:
p<-2
c<-chrs[1]
c
names(pop_files)[p] == "Clades"
paste0("~/data/trd/mapped_reads/ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-pixy_pi.txt.gz")
file.exists(paste0("~/data/trd/mapped_reads/ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-pixy_pi.txt.gz"))

[1] "chromosome1"

[1] FALSE

[1] "~/data/trd/mapped_reads/ALL.DP5-95.chromosome1.DP10.GQ20RGQ20.SNPsRef.vcf.gz-TRD_ChrisC1_1_0.7-pixy_pi.txt.gz"

[1] TRUE

In [41]:
make_bed_file=function(chr,from,to,cross,ID){
    # to get the pi etc values at a precise locus, we need to make a bed file, tab separated
    bed_tbl<-data.table(chr=chr, from=from, to=to)
    bed_out<-paste0("~/data/trd/mapped_reads/pos_for_pixy/",cross,".",ID,".bed")
    fwrite(bed_tbl, bed_out, col.names =FALSE, sep="\t")
    return(bed_out)
}

In [35]:
TRD_loci=fread("/home/jnrunge/data/trd/local_phylogenies_trd_analysis/TRD_regions_with_LP_data.csv.gz")
head(TRD_loci)

ID,lengthSNPs,chr,global_start,global_end,lengthBp,cross,start,end,PCA_eucldist_quantile_1,PCA_eucldist_sd_multiplier_1,IBS_eucldist_quantile_1,IBS_eucldist_sd_multiplier_1,tree_changes_quantile,tree_changes_sd_multiplier,PCA_eucldist_quantile_2,PCA_eucldist_sd_multiplier_2,IBS_eucldist_quantile_2,IBS_eucldist_sd_multiplier_2
<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,155,chromosome2,668344,790905,122561,ChrisC1,444778,567339,0.2969131,-0.2684897,0.7760687,0.13054090,0.002613891,-1.064328,0.3706375,-0.2958096,0.7552774,0.21832468
9,206,chromosome14,9398197,9591625,193428,ChrisC1,224256,417684,0.3225841,-0.2627496,0.4929458,-0.13252233,0.000000000,-1.363081,0.2661504,-0.3312193,0.4711997,-0.17939273
13,815,chromosome15,10015850,10580994,565144,ChrisC1,66814,631958,0.6407128,-0.1538972,0.5699586,-0.07256862,0.000000000,-2.162405,0.6559881,-0.1596002,0.5550016,-0.08543913
14,235,chromosome16,11462227,11683028,220801,ChrisC1,429498,650299,0.4783070,-0.2221963,0.6000849,-0.04848805,0.000000000,-1.508294,0.5172377,-0.2397347,0.6371062,0.01428794
4,2603,chromosome5,2944974,3258307,313333,ChrisC3,65081,378414,0.7185743,-0.1962318,0.4565609,-0.24634727,0.000000000,-1.622491,0.8709027,1.3968420,0.8402461,0.48016916
11,901,chromosome11,6549018,6720267,171249,ChrisC3,18001,189250,0.1547682,-0.3987494,0.5707012,-0.08650142,0.000000000,-1.294373,0.2846080,-0.4131060,0.6197093,-0.33877109


In [45]:
# first, lets compute the values for each TRD locus, no windows

initial_timedate <- Sys.time()
jobname <- "pixy_TRD"
scripts_dir <- "/home/jnrunge/data/trd/mapped_reads/scripts/"

for (p in 1:length(pop_files)) {
  cross <- str_extract(basename(as.character(pop_files[p])), "(?<=_)[[:alnum:]]+(?=_)")
    if (names(pop_files)[p] == "Clades") {
        next
      }
    cross_value<-strsplit(names(pop_files)[p], "_", fixed=TRUE)[[1]][2]
    ID_value<-strsplit(names(pop_files)[p], "_", fixed=TRUE)[[1]][3]
    TRD_locus<-filter(TRD_loci, ID==ID_value & cross==cross_value)
    bed_file<-make_bed_file(pull(TRD_locus, chr), pull(TRD_locus, start), pull(TRD_locus, end), cross_value, ID_value)

    cmd <- paste0(
      "cd ~/data/trd/mapped_reads/ && ",
      "pixy --n_cores 1 --stats dxy fst pi --populations ",
      pop_files[p],  "--bed_file ", bed_file, " --vcf ",
      "ALL.DP5-95.", pull(TRD_locus, chr), ".DP10.GQ20RGQ20.SNPsRef.vcf.gz", " --output_prefix ",
      "ALL.DP5-95.", pull(TRD_locus, chr), ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-theTRDregion-pixy && ",
      "gzip -f ALL.DP5-95.", pull(TRD_locus, chr), ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-theTRDregion-pixy*txt"
    )
    sbatch_list <- execute_complex_sbatch(cmd, jobname = jobname, scripts_dir = scripts_dir, uniqueRunID = paste0(names(pop_files)[p],"_theTRDregion"), cores = "1", mem = "16G", time = "short", env = "bwaetc", initial_timedate = initial_timedate, jobs_simul = 10, jobs_total = 30)

}

[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC1_1_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC1_2_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC1_3_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC1_4_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC3_1_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC3_2_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC4_1_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC5_1_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC5_2_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_ChrisC7_1_0.7_theTRDregion.sbatch"
[1] "/home/jnrunge/data/trd/ma

In [ ]:

for (p in 1:length(pop_files)) {
  cross <- str_extract(basename(as.character(pop_files[p])), "(?<=_)[[:alnum:]]+(?=_)")
  for (c in chrs) {
    if (names(pop_files)[p] == "Clades") {
      if (file.exists(paste0("~/data/trd/mapped_reads/ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-pixy_pi.txt.gz"))) {
        next
      }
    } else {
      if (file.exists(paste0("~/data/trd/mapped_reads/ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-pixy_pi.txt.gz"))) {
        if (file.mtime(paste0("~/data/trd/mapped_reads/ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-pixy_pi.txt.gz")) > file.mtime(paste0("/home/jnrunge/data/TRD/results/shiny/", cross, "-AF.csv.gz.allelesharing.csv.gz"))) {
          next
        }
      }
    }


    cmd <- paste0(
      "cd ~/data/trd/mapped_reads/ && ",
      "pixy --n_cores 2 --stats dxy fst pi --populations ",
      pop_files[p], " --vcf ",
      "ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz", " --output_prefix ",
      "ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-pixy --window_size 1000 && ",
      " gzip -f ALL.DP5-95.", c, ".DP10.GQ20RGQ20.SNPsRef.vcf.gz-", names(pop_files)[p], "-pixy*txt"
    )
    sbatch_list <- execute_complex_sbatch(cmd, jobname = jobname, scripts_dir = scripts_dir, uniqueRunID = paste0(names(pop_files)[p],"_",c), cores = "2", mem = "32G", time = "short", env = "bwaetc", initial_timedate = initial_timedate, jobs_simul = 10, jobs_total = 30)
  }
}

if (exists("sbatch_list") & jobname == "pixy_TRD") {
  print(sbatch_list)
  start_sbatch_list(sbatch_list, 10, jobname, initial_timedate)
}

[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC24_3_0.7_chromosome7.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC24_3_0.7_chromosome13.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC24_3_0.7_chromosome14.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC24_3_0.7_chromosome16.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC24_4_0.7_chromosome5.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC24_4_0.7_chromosome12.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC26_1_0.7_chromosome3.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC26_1_0.7_chromosome6.sbatch"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/pixy_TRD-TRD_YJNRC26_1_0.7_chromosome7.sbatch"
